# Rate performance


   * [GEN Level plots](#Gen-level)
 

In [3]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

In [4]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [5]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [6]:
# %load samples.py
import ROOT

version = 'v44'


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']

sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)

samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]

samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]


tpset_labels = {'DEF': 'NNDR',
                'DEF_em': 'NNDR + EGID',
                'DEF_em_calib': 'NNDR + EGID + calib v0',
                'DEF_pt10': 'NNDR, p_{T}^{L1}>10GeV',
                'DEF_pt10_em': 'NNDR + EGID, p_{T}^{L1}>10GeV',
                'DEF_pt20': 'NNDR, p_{T}^{L1}>20GeV',
                'DEF_pt20_em': 'NNDR + EGID, p_{T}^{L1}>20GeV',
                'DEF_pt25': 'NNDR, p_{T}^{L1}>25GeV',
                'DEF_pt25_em': 'NNDR + EGID, p_{T}^{L1}>25GeV',
                'DEF_pt30': 'NNDR, p_{T}^{L1}>30GeV',
                'DEF_pt30_em': 'NNDR + EGID, p_{T}^{L1}>30GeV'}


particle_labels = {'ele': 'all #eta',
                   'elePt20': 'p_{T}^{GEN}>20GeV',
                   'elePt30': 'p_{T}^{GEN}>30GeV',
                   'elePt40': 'p_{T}^{GEN}>40GeV',
                   'eleA': '|#eta^{GEN}| <= 1.52',
                   'eleB': '1.52 < |#eta^{GEN}| <= 1.7',
                   'eleC': '1.7 < |#eta^{GEN}| <= 2.4',
                   'eleD': '2.4 < |#eta^{GEN}| <= 2.8',
                   'eleE': '|#eta^{GEN}| > 2.8',
                   'eleAB': '|#eta^{GEN}| <= 1.7',
                   'eleABC': '|#eta^{GEN}| <= 2.4',
                   'eleBC': '1.52 < |#eta^{GEN}| <= 2.4',
                   'eleBCD': '1.52 < |#eta^{GEN}| <= 2.8',
                   'eleBCDE': '|#eta^{GEN}| > 1.52',
                   'eleAAA': '|#eta^{GEN}| <= 1.52',
                   'eleAA': '|#eta^{GEN}| <= 1.6',
                   'eleBB': '1.6 < |#eta^{GEN}| <= 2.4',
                   'eleCC': '2.4 < |#eta^{GEN}| <= 2.8',
                   'eleDD': '|#eta^{GEN}| > 2.8',
                   'all': 'all #eta^{L1}',
                   'etaA': '|#eta^{L1}| <= 1.52',
                   'etaB': '1.52 < |#eta^{L1}| <= 1.7)',
                   'etaC': '1.7 < |#eta^{L1}| <= 2.4)',
                   'etaD': '2.4 < |#eta^{L1}| <= 2.8)',
                   'etaE': '|#eta^{L1}| > 2.8',
                   'etaAB': '|#eta^{L1}| <= 1.7',
                   'etaABC': '|#eta^{L1}| <= 2.4',
                   'etaBC': '1.52 < |#eta^{L1}| <= 2.4',
                   'etaBCD': '1.52 < |#eta^{L1}| <= 2.8',
                   'etaBCDE': '|#eta^{L1}| > 1.52'}

samples = []
# particles = ''


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v44.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v44.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v44.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v44.root does not exist


## Electron flat pT gun 2 < pt < 100 GeV

In [8]:

samples = samples_nugunrates

tp_sets = ['DEF', 'DEF_em', 'DEF_em_calib']
particles = ['all', 'etaABC', 'etaBC']
labels = [sample.label for sample in samples]

histo_rate_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)


for tp_set in tp_sets:
    for particle in particles:
        histo_rate_df.loc[tp_set][particle] =  [histos.RateHistos('{}_{}'.format(tp_set, particle), sample.histo_file) for sample in samples]
        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_rate_df   


,all,etaABC,etaBC
DEF,[<RateHistos DEF_all>],[<RateHistos DEF_etaABC>],[<RateHistos DEF_etaBC>]
DEF_em,[<RateHistos DEF_em_all>],[<RateHistos DEF_em_etaABC>],[<RateHistos DEF_em_etaBC>]
DEF_em_calib,[<RateHistos DEF_em_calib_all>],[<RateHistos DEF_em_calib_etaABC>],[<RateHistos DEF_em_calib_etaBC>]


In [9]:
dir(histo_rate_df.loc['DEF', 'all'][0])


['__doc__',
 '__init__',
 '__module__',
 '__repr__',
 'annotateTitles',
 'fill',
 'fill_norm',
 'h_norm',
 'h_pt',
 'h_ptVabseta',
 'name_',
 'normalize',
 'write']

In [10]:
for tp_set in tp_sets:
    for particle in particles:
        for hist in histo_rate_df.loc[tp_set][particle]:
            hist.normalize(31000)

normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


#### 3D clusters matched to GEN

In [11]:

for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_norm for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle])


In [11]:
help(drawSame)

Help on function drawSame in module __main__:

drawSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None, y_axis_label=None, x_axis_label=None, v_lines=None, h_lines=None)



In [12]:

for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_pt for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, options='', text=titles_df.loc[tp_set, particle], logy=True, min_y=1, max_y=70000)


In [25]:
histo_rate_df.loc[['DEF_em', 'DEF_em_calib'], 'all']

DEF_em                [<RateHistos DEF_em_all>]
DEF_em_calib    [<RateHistos DEF_em_calib_all>]
Name: all, dtype: object

In [41]:

for particle in particles:
    print histo_rate_df.loc[['DEF_em', 'DEF_em_calib'], particle]
    hByTPset = [histo[0].h_pt for histo in histo_rate_df.loc[['DEF_em', 'DEF_em_calib'], particle]]
    drawSame(hByTPset, titles_df.loc[['DEF_em', 'DEF_em_calib'], particle], options='', text=particle_labels[particle], logy=True, min_y=1, max_y=70000)


DEF_em                [<RateHistos DEF_em_all>]
DEF_em_calib    [<RateHistos DEF_em_calib_all>]
Name: all, dtype: object
DEF_em                [<RateHistos DEF_em_etaABC>]
DEF_em_calib    [<RateHistos DEF_em_calib_etaABC>]
Name: etaABC, dtype: object
DEF_em                [<RateHistos DEF_em_etaBC>]
DEF_em_calib    [<RateHistos DEF_em_calib_etaBC>]
Name: etaBC, dtype: object


In [37]:
histo_rate_df.loc[tp_set][['etaABC', 'etaBC']]

etaABC    [<RateHistos DEF_etaABC>]
etaBC      [<RateHistos DEF_etaBC>]
Name: DEF, dtype: object

In [40]:
for tp_set in tp_sets:
    print histo_rate_df.loc[tp_set][['etaABC', 'etaBC']]
    hByTPset = [histo[0].h_pt for histo in histo_rate_df.loc[tp_set][['etaABC', 'etaBC']]]
    drawSame(hByTPset, titles_df.loc[tp_set][['etaABC', 'etaBC']], options='', text=tpset_labels[tp_set], logy=True, min_y=1, max_y=70000)


etaABC    [<RateHistos DEF_etaABC>]
etaBC      [<RateHistos DEF_etaBC>]
Name: DEF, dtype: object
etaABC    [<RateHistos DEF_em_etaABC>]
etaBC      [<RateHistos DEF_em_etaBC>]
Name: DEF_em, dtype: object
etaABC    [<RateHistos DEF_em_calib_etaABC>]
etaBC      [<RateHistos DEF_em_calib_etaBC>]
Name: DEF_em_calib, dtype: object


In [13]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptVabseta for histo in histo_rate_df.loc[tp_set, particle]]
        drawSeveral(hByTPset, labels, text=titles_df.loc[tp_set, particle],options='COLZ')


In [14]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle]+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [15]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle]+', p_{T}^{L1}>20GeV', min_y=1, max_y=300, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [16]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle]+', p_{T}^{L1}>30GeV', min_y=0.01, max_y=300, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [17]:
for particle in particles:
    for tp_set in tp_sets:
        hByTPset = [histo.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for histo in histo_rate_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, text=titles_df.loc[tp_set, particle]+', p_{T}^{L1}>40GeV', min_y=0.09, max_y=300, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
